In [1]:
import tech_lib as tech
import binance_api as binance
%load_ext autoreload
%autoreload 2
import itertools
import pandas as pd
import numpy as np
import winsound
import os
from datetime import datetime
import torch
print(f'Cuda available: {torch.cuda.is_available()}')
print(f'Cuda version: {torch.version.cuda}')
print(f'Cuda device name: {torch.cuda.get_device_name(0)}')

Cuda available: True
Cuda version: 12.6
Cuda device name: NVIDIA GeForce RTX 4060


In [2]:
api_key = 'lmxX1ymc6vGTQYkgUR194NeUxrhZQXM2xnf0rvjtQV4XxgP9xS9rawF6CvQRO2pw'
api_secret = 'rNFCb6OYjktZ1O6yfpNy1EItYJGwOMe8rtW8salpdXDHaNQGdxJDwpqgXrysxjN7'

ticker = 'BTCUSDT'
interval = '1m'
start_date = (2024, 6, 15, 0, 0, 0)
end_date = (2025, 1, 15, 0, 0, 0)

start_date_str = datetime(*start_date).strftime('%Y%m%d')
end_date_str = datetime(*end_date).strftime('%Y%m%d')
dataset_path = os.getcwd()+f'\\dataset\\{ticker}-{interval}-{start_date_str}-{end_date_str}.csv'

if os.path.exists(dataset_path):
    dataset = pd.read_csv(dataset_path, index_col='Open Time')
else:
    dataset = binance.historical_data(
        api_key,
        api_secret,
        ticker,
        interval,
        start_date,
        end_date
    )
    dataset = dataset.drop(['Ignore', 'Close Time'], axis=1)

dataset.to_csv(dataset_path)

In [3]:
#Calcul des outputs
dataset['Close_target'] = dataset['Close'].shift(-1)
dataset['DirVar_target'] = pd.Series([1 if diff > 0 else 0 for diff in dataset['Close'].diff(1)], index=dataset.index, dtype=int).shift(-1)
#dataset['Returns'] = np.log(dataset['Close'] / dataset['Close'].shift(1))
#dataset['Returns_target'] = dataset.Returns - dataset.Returns.diff(-1)

# Boucle pour calculer les indicateurs avec différentes fenêtres
for window in range(30, 121, 30):
    dataset[f'cmf_{window}'] = tech.cmf(high=dataset.High, low=dataset.Low, close=dataset.Close, volume=dataset.Volume, window=window)
    dataset[f'rsi_{window}'] = tech.rsi(close=dataset.Close, window=window)
    dataset[f'ma_{window}'] = tech.ma(close=dataset.Close, window=window)
    dataset[f'ema_{window}'] = tech.ema(close=dataset.Close, window=window)
    dataset[f'atr_{window}'] = tech.atr(high=dataset.High, low=dataset.Low, close=dataset.Close, window=window)
    dataset[f'bollinger_{window}'] = tech.bollinger_band_width(close=dataset.Close, window=window, num_std=2)
dataset['obv'] = tech.obv(dataset.Close, dataset.Volume)

dataset = dataset.dropna()

In [ ]:
from gru_lib import GRUModel, Pipeline
%load_ext autoreload
%autoreload 2

# Définir les plages de valeurs pour les hyperparamètres
period_size = 100000
hidden_size = 8
num_layers = 1
dropout = 0.0
lr = 0.1
batch_size = 512
epochs = 5
num_workers = 4
train_size = 0.7
val_size = 0.15
test_size = 0.15
seq_size = 30


inputs = [
    'Open', 'Close', 'High', 'Low', 
    'Volume', 'Number of Trades', 'Taker Buy Base Asset Volume',
    'obv', 'cmf_30', 'ema_30', 'atr_30',
    'rsi_30', 'ma_30', 'bollinger_30',
]
min_imputs = 6
outputs = 'DirVar_target'

result_path = os.getcwd()+f'\\train_result\\GRU-{ticker}-{interval}-inputs_selection.csv'

# Générer toutes les combinaisons additives possibles des inputs
input_combinations = [list(itertools.combinations(inputs, r)) for r in range(min_imputs, len(inputs) + 1)]
input_combinations = [item for sublist in input_combinations for item in sublist]

counter = 0
for input_subset in input_combinations:
    time_lauch = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'---------- Launch Time: {time_lauch:<10} ---------- Progress: {(counter):,}/{(len(input_combinations)):,} ----------')

    # Garder la période étudiée
    dataset_subset = dataset.iloc[period_size:]
    dataset_subset = dataset_subset.dropna()

    # Initialiser le modèle
    model = GRUModel(
        input_size=len(input_subset),
        output_size=1,
        hidden_size=hidden_size, 
        num_layers=num_layers,
        dropout=dropout
    )

    # Initialiser le pipeline
    pipeline = Pipeline(
        model=model, 
        dataset=dataset_subset,
        inputs=list(input_subset),
        outputs=outputs
    )

    # Définir les hyperparamètres
    pipeline.hyper_param(
        lr=lr, 
        batch_size=batch_size, 
        epochs=epochs,
        num_workers=num_workers
    )

    # Prétraitement des données
    pipeline.preprocess(
        train_size=train_size,
        val_size=val_size, 
        test_size=test_size, 
        seq_size=seq_size
    )

    pipeline.train()
    pipeline.eval()
    pipeline.save(result_path)

    print("\n")
    counter += 1


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
---------- Launch Time: 2025-01-22 20:35:54 ---------- Progress: 0/166,693,920 ----------
Epoch 1/5, Train Loss: 6,432,548,305.291228, Validation Loss: 30,053,709,295.213116, Time: 0:00:01.785168


KeyboardInterrupt: 

In [ ]:
from gru_lib import GRUModel, Pipeline
%load_ext autoreload
%autoreload 2

# Définir les plages de valeurs pour les hyperparamètres
param_grid = {
    "period_size"   : [100000],
    "hidden_size"   : [32],
    "num_layers"    : [4],
    "dropout"       : [0.1],
    "lr"            : [0.01],
    "batch_size"    : [128],
    "epochs"        : [50],
    "num_workers"   : [4],
    "delta"         : [1],
    "weight_decay"  : [1e-4],
    "train_size"    : [0.7],
    "val_size"      : [0.15],
    "test_size"     : [0.15],
    "seq_size"      : [60]
}

inputs = [
    'High', 'Low', 'Open', 'Close',
    'Volume', 'obv', 'cmf_90', 'ma_90'
]
outputs = 'Close_target'

result_path = os.getcwd()+f'\\train_result\\GRU-{ticker}-{interval}-param_selection.csv'

# Générer toutes les combinaisons possibles d'hyperparamètres
param_combinations = list(itertools.product(*param_grid.values()))

counter = 0
for params in param_combinations:
    time_lauch = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f'---------- Launch Time: {time_lauch:<10} ---------- Progress: {(counter):,}/{(len(param_combinations)):,} ----------')

    # Extraire les paramètres pour cette combinaison
    hyperparams = dict(zip(param_grid.keys(), params))

    # Garder la période étudiée
    dataset_subset = dataset.iloc[-hyperparams["period_size"]:]
    dataset_subset = dataset_subset.dropna()

    # Initialiser le modèle
    model = GRUModel(
        input_size=len(inputs),
        output_size=1,
        hidden_size=hyperparams["hidden_size"], 
        num_layers=hyperparams["num_layers"],
        dropout=hyperparams["dropout"]
    )

    # Initialiser le pipeline
    pipeline = Pipeline(
        model=model, 
        dataset=dataset_subset,
        inputs=inputs,
        outputs=outputs
    )

    # Définir les hyperparamètres
    pipeline.hyper_param(
        lr=hyperparams["lr"], 
        batch_size=hyperparams["batch_size"], 
        epochs=hyperparams["epochs"],
        num_workers=hyperparams["num_workers"],
        delta=hyperparams["delta"],
        weight_decay=hyperparams["weight_decay"]
    )

    # Prétraitement des données
    pipeline.preprocess(
        train_size=hyperparams["train_size"],
        val_size=hyperparams["val_size"], 
        test_size=hyperparams["test_size"], 
        seq_size=hyperparams["seq_size"]
    )

    pipeline.train()
    pipeline.eval()
    pipeline.save(result_path)

    pipeline.loss(plot=True)
    pipeline.pred(plot=True)

    print("\n")
    counter += 1

winsound.Beep(1000, 500)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
---------- Launch Time: 2025-01-23 21:24:42 ---------- Progress: 0/1 ----------
Epoch 1/50, Train Loss: 94,887.478733, Validation Loss: 439,859.753112, Time: 0:00:40.325395
Epoch 2/50, Train Loss: 94,706.099791, Validation Loss: 439,018.954118, Time: 0:00:41.902652
Epoch 3/50, Train Loss: 94,526.746401, Validation Loss: 438,187.544756, Time: 0:00:46.401106
Epoch 4/50, Train Loss: 94,344.843422, Validation Loss: 437,344.316539, Time: 0:00:42.639538
Epoch 5/50, Train Loss: 94,164.386883, Validation Loss: 436,507.793432, Time: 0:00:41.254528
Epoch 6/50, Train Loss: 93,985.909821, Validation Loss: 435,680.446372, Time: 0:00:41.186472
Epoch 7/50, Train Loss: 93,805.239788, Validation Loss: 434,842.933594, Time: 0:00:41.018807
Epoch 8/50, Train Loss: 93,626.281350, Validation Loss: 434,013.355072, Time: 0:00:41.457772
Epoch 9/50, Train Loss: 93,445.646509, Validation Loss: 433,176.005429, Time: 0:00:40.74

AttributeError: 'GRUModel' object has no attribute 'hidden_size'